In [ ]:
# 目标 构建语义搜索引擎
# 1.文档和文档加载
# 2.文本拆分器使用
# 3.嵌入模型使用
# 4.向量存储和召回（就是根据语义检索相似文档块）

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
# 依赖安装
%pip install langchain-community pypdf

In [ ]:
# 相关环境变量设置
import config_loader

config_loader.load_env()

In [ ]:
# 文档和文档加载器
# 文档（Document）是 langchain 的一个抽象概念，代表文本单元和相关元数据，有三个属性
# page_content 文档内容字符串
# metadata 包含任意元数据的字典，比如说获取相关文档来源，与其他文档的关系及其它额外信息
# id 文档的字符串标识符
# 一般单个 Document 对象代表较大文档的一部分
# 文档使用示例
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
]
documents

In [ ]:
# 加载pdf文件为 Document 对象
# 使用基于 pypdf 的 pdf 加载器 pdf一个页面会加载成为一个 Document 对象
from langchain_community.document_loaders import PyPDFLoader

file_path = "../data/Kafka权威指南-22-52.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))
print('='*100)
# document 对象有原始文档字符串和元数据
print(f"{docs[0].page_content[:200]}\n")
print(docs[0].metadata)

In [ ]:
# 对于信息检索来说，一个页面一个 Document 太粗略了，所以需要进一步细致的拆分
# 这里就用上了文本分割器 RecursiveCharacterTextSplitter，这个分割器是使用常用分隔符（比如说换行符）递归分割文档，直到每个块的大小合适
# 这也是针对一般文本用例推荐的分割器
# 我们将文档分割成1000个字符的块，块之间有200个字符的重叠，重叠有助于减轻将语句与与其相关的重要上下文分离的问题
# 我们设置 add_start_index=True 每个分割文档在初始文档中开始的字符索引作为元数据属性 “start_index” 保存。
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
# 基于上面的文档集合 再次通过文本拆分成更多的小文档块 返回一个 List[Document]
all_splits = text_splitter.split_documents(docs)

print(type(all_splits[0]))
print(len(all_splits))
print("="*100)
print(f'第一个chunk：{all_splits[0]}')

In [ ]:
# 嵌入模型使用
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# 初始化
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

# 使用嵌入模型 将文本块生成向量嵌入 List[float]
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"生成的向量嵌入长度 {len(vector_1)}\n")
print(vector_1[:10])

In [ ]:
# 向量存储 有多种方式可以内存，也可以用 FAISS 第三方向量库 这里方便起见 用内存
from langchain_core.vectorstores import InMemoryVectorStore

# 初始化向量存储
vector_store = InMemoryVectorStore(embeddings)
# 把文档做索引操作 加到库中
ids = vector_store.add_documents(documents=all_splits)
# 语义检索 返回语义相似的文档对象list
results = vector_store.similarity_search(
    "Kafka Broker 配置"
)

print(len(results))

print(results)
print(results[0])

# 返回带分数的
results = vector_store.similarity_search_with_score(
    "Kafka Broker 配置"
)
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

In [ ]:
# 根据向量嵌入来查询相似性
query_embedding = embeddings.embed_query("Kafka Broker 配置")
results = vector_store.similarity_search_with_score_by_vector(query_embedding)

print(len(results))
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)